# Sora with Azure AI Foundry

Sora is an AI model from OpenAI that can create realistic and imaginative video scenes from text instructions. The model is capable of generating a wide range of video content, including realistic scenes, animations, and special effects. Several video resolutions and durations are supported.

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install moviepy


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 217.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.9 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 269.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.7 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/129.9 kB ? eta -:--:--
   ------------------------------------- -- 122.9/129.9 kB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 129.9/129.9 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/317.6 kB ? eta -:--:--
   -----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import base64
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import Video, FileLink
from moviepy import *

In [8]:
sys.version

'3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]'

In [9]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Dec-2025 22:38:16


## Settings

In [10]:
OUTPUT_DIR = "videos"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
load_dotenv("azure.env")

endpoint = ""
api_key = ""

model = "sora"

# Function

In [12]:
def sora(prompt, width=480, height=480, n_seconds=5):
    """
    Generates a video based on the given prompt using the SORA model.

    Parameters:
    prompt (str): The text prompt to generate the video.
    width (int): The width of the video. Supported values are 480, 854, 720, 1080, and 1920.
    height (int): The height of the video. Supported values are 480, 854, 720, 1080, and 1920.
    n_seconds (int): The duration of the video in seconds. Must be between 1 and 20 seconds.
    n_variants (int): The number of video variants to generate.
    
    Returns:
    str: The filename of the generated video.

    Raises:
    Exception: If the video generation job fails or no generations are found.
    """
    start = time.time()

    api_version = 'preview'
    headers = {"api-key": api_key, "Content-Type": "application/json"}

    idx = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')
    suffix = prompt[:30].replace(",", "_").replace(".", "_").replace(" ", "_")
    output_filename = os.path.join(OUTPUT_DIR, f"sora_{idx}_{suffix}.mp4")

    # 1. Create a video generation job
    create_url = f"{endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    
    body = {
        "prompt": prompt,
        "width": width,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "height": height,  # 480x480, 480x854, 854x480, 720x720, 720x1280, 1280x720, 1080x1080, 1080x1920, 1920x1080.
        "n_seconds": n_seconds,  # between 1 and 20 seconds
        "model": model,  # SORA model
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()

    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Full response JSON:", response.json())
    print()

    job_id = response.json()["id"]
    now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
    print(f"{now} Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status = None

    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
        print(f"{now} Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])

        if generations:
            now = datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')
            print(f"\n{now} ✅ Done. Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)

            if video_response.ok:
                # Downloading the video
                print("\nDownloading the video...")
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f"SORA Generated video saved: '{output_filename}'")

                elapsed = time.time() - start
                minutes, seconds = divmod(elapsed, 60)
                print(f"Done in {minutes:.0f} minutes and {seconds:.0f} seconds")

                return output_filename
        else:
            raise Exception("Error. No generations found in job result.")
    else:
        raise Exception(f"Error. Job did not succeed. Status: {status}")

## Example

In [13]:
prompt = "A young boy and his father playing together in the ocean on the beach."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

17-Dec-2025 22:40:50 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr316q2eme86g0cn4xh76ks', 'status': 'preprocessing', 'created_at': 1766040050, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A young boy and his father playing together in the ocean on the beach.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:40:50 Job created: task_01kcr316q2eme86g0cn4xh76ks
17-Dec-2025 22:40:57 Job status: preprocessing
17-Dec-2025 22:41:04 Job status: queued
17-Dec-2025 22:41:11 Job status: running
17-Dec-2025 22:41:18 Job status: processing
17-Dec-2025 22:41:25 Job status: processing
17-Dec-2025 22:41:34 Job status: succeeded

17-Dec-2025 22:41:34 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos\sora_17Dec2025_224047_A_young_boy_and_his_father_pla.mp4'
Done in 0 minutes and 55 seconds


In [14]:
Video(generated_video)

In [15]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224047_A_young_boy_and_his_father_pla.mp4

## Another example

In [16]:
prompt = "A close up view of a glass sphere that has a zen garden within it. There is a small dwarf in the sphere who is raking the zen garden and creating patterns in the sand."

generated_video = sora(prompt, width=480, height=480, n_seconds=5)

17-Dec-2025 22:43:03 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr358x4eayrfr01x2h5ddwx', 'status': 'preprocessing', 'created_at': 1766040183, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A close up view of a glass sphere that has a zen garden within it. There is a small dwarf in the sphere who is raking the zen garden and creating patterns in the sand.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 480, 'width': 480, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:43:03 Job created: task_01kcr358x4eayrfr01x2h5ddwx
17-Dec-2025 22:43:10 Job status: preprocessing
17-Dec-2025 22:43:17 Job status: running
17-Dec-2025 22:43:24 Job status: processing
17-Dec-2025 22:43:30 Job status: processing
17-Dec-2025 22:43:38 Job status: succeeded

17-Dec-2025 22:43:38 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos\sora_17Dec2025_224301_A_close_up_view_of_a_glass_sph.mp4'
Done in 0 minutes and 42

In [17]:
Video(generated_video)

In [18]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224301_A_close_up_view_of_a_glass_sph.mp4

## Another example

In [19]:
prompt = "Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

17-Dec-2025 22:43:54 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr36tpsf4ptakecy19zv9qf', 'status': 'preprocessing', 'created_at': 1766040234, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Several giant wooly mammoths approach treading through a snowy meadow, their long wooly fur lightly blows in the wind as they walk, snow covered trees and dramatic snow capped mountains in the distance, mid afternoon light with wispy clouds and a sun high in the distance creates a warm glow, the low camera view is stunning capturing the large furry mammal with beautiful photography, depth of field.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:43:54 Job created: task_01kcr36tpsf4ptakecy19zv9qf
17-Dec-2025 22:44:02 Job status: queued
17-Dec-2025 22:44:09 Job status: queued
17-Dec-2025 22:44:16 Job status: running
17-Dec-2025 22:44:23 Job status: running

In [20]:
Video(generated_video, width=640)

In [21]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224352_Several_giant_wooly_mammoths_a.mp4

## Another example

In [22]:
prompt = "Drone view of waves crashing against the rugged cliffs along Big Sur's garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff's edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff's edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=15,
                       )

17-Dec-2025 22:45:17 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr39bjkfk8bvtnvt6nvapp3', 'status': 'preprocessing', 'created_at': 1766040317, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Drone view of waves crashing against the rugged cliffs along Big Sur's garay point beach. The crashing blue waters create white-tipped waves, while the golden light of the setting sun illuminates the rocky shore. A small island with a lighthouse sits in the distance, and green shrubbery covers the cliff's edge. The steep drop from the road down to the beach is a dramatic feat, with the cliff's edges jutting out over the sea. This is a view that captures the raw beauty of the coast and the rugged landscape of the Pacific Coast Highway.", 'model': 'sora', 'n_variants': 1, 'n_seconds': 15, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:45:17 Job created: task_01kcr39bjkfk8bvtnvt6nvapp3
17-Dec-2025 22:45:24 

In [23]:
Video(generated_video)

In [24]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224514_Drone_view_of_waves_crashing_a.mp4

## Another example

In [25]:
prompt = "Extreme close up of a 25 year old woman's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

17-Dec-2025 22:48:18 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr3ew29faarf3178ya8xprr', 'status': 'preprocessing', 'created_at': 1766040498, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Extreme close up of a 25 year old woman's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:48:18 Job created: task_01kcr3ew29faarf3178ya8xprr
17-Dec-2025 22:48:25 Job status: preprocessing
17-Dec-2025 22:48:32 Job status: queued
17-Dec-2025 22:48:38 Job status: running
17-Dec-2025 22:48:45 Job status: running
17-Dec-2025 22:48:52 Job status: running
17-Dec-2025 22:48:59 Job status: processing
17-Dec-2025 22:49:05 Job status: processing
17-Dec-2025 22:49:12 Job status: succeeded

17-Dec-2025 22:49:12 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos\so

In [26]:
Video(generated_video, width=860)

In [27]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224815_Extreme_close_up_of_a_25_year_.mp4

## Another example

In [28]:
prompt = "Extreme close up of a 50 year old male's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic"

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=5,
                       )

17-Dec-2025 22:49:41 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr3hdhbfqbvw9pj2czam1a1', 'status': 'preprocessing', 'created_at': 1766040581, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': "Extreme close up of a 50 year old male's eye blinking, cinematic film shot in 70mm, depth of field, vivid colors, cinematic", 'model': 'sora', 'n_variants': 1, 'n_seconds': 5, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:49:41 Job created: task_01kcr3hdhbfqbvw9pj2czam1a1
17-Dec-2025 22:49:48 Job status: queued
17-Dec-2025 22:49:55 Job status: queued
17-Dec-2025 22:50:01 Job status: running
17-Dec-2025 22:50:08 Job status: running
17-Dec-2025 22:50:15 Job status: processing
17-Dec-2025 22:50:22 Job status: processing
17-Dec-2025 22:50:30 Job status: succeeded

17-Dec-2025 22:50:30 ✅ Done. Video generation succeeded.

SORA Generated video saved: 'videos\sora_17Dec2025_224939_Extreme_close_up_of_a_50_year

In [29]:
Video(generated_video, width=860)

In [30]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_224939_Extreme_close_up_of_a_50_year_.mp4

## Another example

In [31]:
prompt = "Reflections in the window of a train traveling through the Tokyo suburbs."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=10,
                       )

17-Dec-2025 22:51:41 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr3mzw1fny8jj9zp5gfty4s', 'status': 'preprocessing', 'created_at': 1766040698, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Reflections in the window of a train traveling through the Tokyo suburbs.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 10, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:51:41 Job created: task_01kcr3mzw1fny8jj9zp5gfty4s
17-Dec-2025 22:51:47 Job status: queued
17-Dec-2025 22:51:54 Job status: running
17-Dec-2025 22:52:01 Job status: running
17-Dec-2025 22:52:08 Job status: running
17-Dec-2025 22:52:15 Job status: running
17-Dec-2025 22:52:22 Job status: running
17-Dec-2025 22:52:28 Job status: running
17-Dec-2025 22:52:35 Job status: running
17-Dec-2025 22:52:42 Job status: processing
17-Dec-2025 22:52:49 Job status: processing
17-Dec-2025 22:52:55 Job status: succeeded

17-Dec-2025 22:52:55 ✅ Done. 

In [32]:
Video(generated_video, width=860)

In [33]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_225136_Reflections_in_the_window_of_a.mp4

## Another example

In [34]:
prompt = "A drone view of flock of paper airplanes multicolors flutters through a dense jungle, weaving around trees as if they were migrating birds."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

17-Dec-2025 22:53:43 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr3rqr1e5svntajz57rshnc', 'status': 'preprocessing', 'created_at': 1766040821, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'A drone view of flock of paper airplanes multicolors flutters through a dense jungle, weaving around trees as if they were migrating birds.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 7, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:53:43 Job created: task_01kcr3rqr1e5svntajz57rshnc
17-Dec-2025 22:53:49 Job status: queued
17-Dec-2025 22:53:57 Job status: running
17-Dec-2025 22:54:03 Job status: running
17-Dec-2025 22:54:10 Job status: running
17-Dec-2025 22:54:17 Job status: running
17-Dec-2025 22:54:24 Job status: running
17-Dec-2025 22:54:30 Job status: processing
17-Dec-2025 22:54:37 Job status: processing
17-Dec-2025 22:54:44 Job status: succeeded

17-Dec-2025 22:54:44 ✅ Done. Video generation 

In [35]:
Video(generated_video, width=860)

In [36]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_225338_A_drone_view_of_flock_of_paper.mp4

## Another exemple

In [37]:
prompt = "Street of Tokyo, rainy day with people walking in a street, restaurants in the street."

generated_video = sora(prompt,
                       width=1280,
                       height=720,
                       n_seconds=7,
                       )

17-Dec-2025 22:55:46 Full response JSON: {'object': 'video.generation.job', 'id': 'task_01kcr3wha6fqebtkjeg2r4z2w4', 'status': 'preprocessing', 'created_at': 1766040945, 'finished_at': None, 'expires_at': None, 'generations': [], 'prompt': 'Street of Tokyo, rainy day with people walking in a street, restaurants in the street.', 'model': 'sora', 'n_variants': 1, 'n_seconds': 7, 'height': 720, 'width': 1280, 'inpaint_items': None, 'failure_reason': None}

17-Dec-2025 22:55:46 Job created: task_01kcr3wha6fqebtkjeg2r4z2w4
17-Dec-2025 22:55:52 Job status: queued
17-Dec-2025 22:55:59 Job status: queued
17-Dec-2025 22:56:06 Job status: running
17-Dec-2025 22:56:12 Job status: running
17-Dec-2025 22:56:19 Job status: running
17-Dec-2025 22:56:26 Job status: running
17-Dec-2025 22:56:33 Job status: processing
17-Dec-2025 22:56:40 Job status: processing
17-Dec-2025 22:56:46 Job status: processing
17-Dec-2025 22:56:53 Job status: succeeded

17-Dec-2025 22:56:53 ✅ Done. Video generation succeeded.

In [38]:
Video(generated_video, width=860)

In [39]:
video_link = FileLink(path=generated_video)
video_link

c:\Users\Student\Desktop\Lab Files\sora\videos\sora_17Dec2025_225543_Street_of_Tokyo__rainy_day_wit.mp4

In [40]:
!ls $OUTPUT_DIR -lh

'ls' is not recognized as an internal or external command,
operable program or batch file.
